In [69]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
#jupyter内で画像を表示
%matplotlib inline 
#別ウィンドウで画像表示
# %matplotlib qt 
import csv
import json
from ipywidgets import interact, interactive, fixed, RadioButtons
import ipywidgets as widgets
from IPython.display import display
import pathlib
seed = '10005'
node_num = '400'
shadow = 'shadow20_2'
sigo_path = pathlib.Path('../../get_data/Horizontal_axis/' + shadow + '/sigo/sigo-seed_' +seed +'nodenum_' +node_num+'.csv')
lsgo_path = pathlib.Path('../../get_data/Horizontal_axis/' + shadow + '/lsgo/lsgo-seed_' +seed +'nodenum_' +node_num+'.csv')
# sigo_path = pathlib.Path('../../ns3.30/data/no_buildings/sigo/sigo-seed_' +seed +'nodenum_' +node_num+'.csv')
# lsgo_path = pathlib.Path('../../ns3.30/data/no_buildings/lsgo/lsgo-seed_' +seed +'nodenum_' +node_num+'.csv')

# csvファイルの取得
pandasを利用：https://note.nkmk.me/python-pandas-value-counts/

In [70]:
sigo_df = pd.read_csv(sigo_path)
lsgo_df = pd.read_csv(lsgo_path)

In [71]:
print(sigo_path)
print(lsgo_path)

..\..\get_data\Horizontal_axis\shadow20_2\sigo\sigo-seed_10005nodenum_400.csv
..\..\get_data\Horizontal_axis\shadow20_2\lsgo\lsgo-seed_10005nodenum_400.csv


# destination_idごとのパケット数

In [72]:
lsgo_df['destination_id'].value_counts().to_dict()

{17: 98,
 98: 45,
 235: 41,
 199: 40,
 170: 39,
 123: 38,
 7: 34,
 263: 24,
 368: 18,
 270: 16}


destination_id を重複なしにリスト保存

In [73]:
sigo_df['destination_id'].value_counts().to_dict()

{199: 98,
 17: 70,
 98: 64,
 263: 56,
 270: 55,
 235: 46,
 7: 42,
 170: 36,
 123: 36,
 368: 15}

In [74]:
dest_list =lsgo_df['destination_id'].value_counts().index.tolist()
dest_list.sort()
len(dest_list)

10

In [75]:
dest_list = sigo_df['destination_id'].value_counts().index.tolist()
dest_list.sort()
len(dest_list)

10

sigoとlsgoそれぞれ目的地に届いたidだけをlistに保存

In [76]:
sigo_reach_list = sigo_df[sigo_df['recv_id'] == sigo_df['destination_id']]['destination_id'].value_counts().index.tolist()
lsgo_reach_list = lsgo_df[lsgo_df['recv_id'] == lsgo_df['destination_id']]['destination_id'].value_counts().index.tolist()
sigo_reach_list.sort()
lsgo_reach_list.sort()

print('sigo:',sigo_reach_list)
print('lsgo:',lsgo_reach_list)

sigo: [7, 17, 98, 123, 170, 199, 235, 263, 270]
lsgo: [17, 98, 123, 170, 199]


# 平均delayを返すメソッド

In [77]:
def packet_delay(path, spefic=0): #spefic 0 or id   0の時平均を返す idのときそのdestination id のdelayを返す
    delay_list = []
    #csvファイルの取得
    path_df = pd.read_csv(path)
    #目的地に届いたidだけをlistに保存
    reach_list = path_df[path_df['recv_id'] == path_df['destination_id']]['destination_id'].value_counts().index.tolist()
    if(spefic == 0):
        #reach_listを回す
        for reach_id in reach_list:
            source_time = path_df[path_df['destination_id'] == reach_id]['time']
            recv_time = path_df[(path_df['destination_id'] == reach_id) & (path_df['destination_id'] == path_df['recv_id'])]['time']
            delay = int(recv_time.head(1).iloc[-1]) - int(source_time.head(1).iloc[-1])
            delay = delay / 100000
        #         print('desID:', reach_id, 'delay:',delay)
            delay_list.append(delay)
        delay_mean = np.mean(delay_list)
        return delay_mean
    else:
        source_time = path_df[path_df['destination_id'] == spefic]['time']
        recv_time = path_df[(path_df['destination_id'] == spefic) & (path_df['destination_id'] == path_df['recv_id'])]['time']
        delay = int(recv_time.head(1).iloc[-1]) - int(source_time.head(1).iloc[-1])
        delay = delay / 100000
        return delay

In [78]:
packet_delay(sigo_path)

0.28622777777777775

In [79]:
packet_delay(lsgo_path)

0.318564

# 軌跡描画

In [80]:
def draw_source_area(log, img_area):
    cv2.rectangle(img_area, (log.source_x - 15, log.source_y - 15), (log.source_x + 15, log.source_y + 15), (255, 255, 0),thickness=8)
    cv2.rectangle(img_area, (log.destination_x - 15, log.destination_y - 15), (log.destination_x + 15, log.destination_y + 15), (255, 255, 0),thickness=8)
    #source nodeを視覚で囲む
    cv2.rectangle(img_area, (log.source_x - 15, log.source_y - 15), (log.source_x + 15, log.source_y + 15), (255, 255, 0),thickness=8)

    #方眼
    plt.figure(figsize=(10, 10))
    print('グラフサイズ')
    plt.grid(color='w', linestyle='dotted', linewidth=1.0)
    plt.xlim([0, 1000])
    plt.xticks([1000,800,600,400,200, 0]) 
    plt.ylim([0, 1000])
    plt.yticks([1000,800,600,400,200, 0])  #メモリの間隔を指定 
    print('source_area 描画', log.source_x)

# パケットの軌跡を描画
def draw(log,count,img_area,protocol,text):
    if protocol == 'sigo':
        line_color = (0, 255, 0)
        thick = 7
        des_thick = 8
    else:
        line_color = (255, 0, 0)
        thick = 2
        des_thick = 2
    
    cv2.arrowedLine(img_area,(log.source_x, log.source_y), 
                            (log.recv_x, log.recv_y), line_color, thickness=thick, tipLength=0.1)
    cv2.circle(img=img_area, center=(log.source_x,
                           log.source_y), radius=5, color=(255,0,0), thickness=-1)
    cv2.circle(img=img_area, center=(log.recv_x,
                           log.recv_y), radius=5, color=(255,0,0), thickness=-1)
    #受け取ったのが宛先だったならば
    if log.recv_id == log.destination_id:
        cv2.drawMarker(img_area, (log.recv_x, log.recv_y), line_color, thickness=des_thick, markerType=cv2.MARKER_TILTED_CROSS, markerSize=25)
    if text == 'あり':    
        cv2.putText(img_area, str(log.source_id), (log.source_x + 10,log.source_y +10 ), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), thickness=2)
        cv2.putText(img_area, str(log.recv_id), (log.recv_x + 10,log.recv_y +10 ), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), thickness=2)

In [81]:
img_area = np.zeros(shape=(1000,1000, 3), dtype=np.int16)
protocol_list = ["both","lsgo", "sigo"]
text_list = ["なし", "あり"]
print('sigo:',sigo_reach_list,'PDR:',len(sigo_reach_list)/len(sigo_df['destination_id'].value_counts().index.tolist()))
print('lsgo:',lsgo_reach_list,'PDR:',len(lsgo_reach_list)/len(lsgo_df['destination_id'].value_counts().index.tolist()))
print('sigo delay:', packet_delay(sigo_path))
print('lsgo delay:', packet_delay(lsgo_path))

#packetの軌跡を描画 
@interact(minHop=(0,19,1), maxHop=(1,21, 1), desId=dest_list, protocol = protocol_list, text = text_list)
def packetDraw(protocol, desId, minHop=0, maxHop = 21, text = "なし"):
    
    img_area = np.zeros(shape=(2100,2100, 3), dtype=np.int16)
    count = 1
    if protocol == 'sigo':
        
#         print('sigo packet delay id', desId, 'delay', packet_delay(sigo_path, desId))
        
        for log in sigo_df[sigo_df['destination_id'] == desId].itertuples():
            if count == 1:
                draw_source_area(log, img_area)
                count = count +1
            if log.hopcount >= minHop and log.hopcount <= maxHop:
                draw(log,count,img_area,protocol,text)
#                 count = count +1
                
        plt.imshow(img_area)
    elif protocol == 'lsgo':
        
#         print('lsgo packet delay id', desId, 'delay', packet_delay(lsgo_path, desId))
        
        for log in lsgo_df[lsgo_df['destination_id'] == desId].itertuples():
            if count == 1:
                draw_source_area(log, img_area)
                count = count +1
                
            if log.hopcount >= minHop and log.hopcount <= maxHop:
                draw(log,count,img_area,protocol,text)
#                 count = count +1
                
        plt.imshow(img_area)
    else:
        
        if(desId in sigo_reach_list):
            print('sigo packet delay id', desId, 'delay', packet_delay(sigo_path, desId))
        if(desId in lsgo_reach_list):
            print('lsgo packet delay id', desId, 'delay', packet_delay(lsgo_path, desId))
        
        for log in sigo_df[sigo_df['destination_id'] == desId].itertuples():
            if count == 1:
                draw_source_area(log, img_area)
                count = count +1
                
            if log.hopcount >= minHop and log.hopcount <= maxHop:
                draw(log,count,img_area,'sigo',text)
#                 count = count +1
        for log in lsgo_df[lsgo_df['destination_id'] == desId].itertuples():
            if log.hopcount >= minHop and log.hopcount <= maxHop:
                draw(log,count,img_area,'lsgo',text)
#                 count = count +1
        plt.imshow(img_area)

sigo: [7, 17, 98, 123, 170, 199, 235, 263, 270] PDR: 0.9
lsgo: [17, 98, 123, 170, 199] PDR: 0.5
sigo delay: 0.28622777777777775
lsgo delay: 0.318564


interactive(children=(Dropdown(description='protocol', options=('both', 'lsgo', 'sigo'), value='both'), Dropdo…

210 344

In [82]:
def output_df(hopcount, des_id):
    print('lsgo')
    display(lsgo_df[(lsgo_df['hopcount'] == hopcount) & (lsgo_df['destination_id'] == des_id)])
    print('sigo')
    display(sigo_df[(sigo_df['hopcount'] == hopcount) & (sigo_df['destination_id'] == des_id)])

In [314]:
@interact(hopcount=(0,21,1), des_id = dest_list)
def df_run(hopcount, des_id):
    output_df(hopcount, des_id)

interactive(children=(IntSlider(value=10, description='hopcount', max=21), Dropdown(description='des_id', opti…